In [1]:
import pandas as pd
import sqlite3

What you need to do in this exercise:

- create a connection to the database using the library sqlite3
- get the schema of the table test
- get only the first 10 rows of the table test to check what the table looks like
- find among all the users the minimum value of the delta between the first commit
of the user and the deadline of the corresponding lab using only one query

    - do this by joining the table with the table deadlines
    the difference should be displayed in hours
    - do not take the lab ’project1’ into account, it has longer deadlines and will be
    an outlier
    - the value should be stored in the dataframe df_min with the corresponding
    uid


- do the same thing, but for the maximum, using only one query, the dataframe name is
df_max
- do the same thing but for the average, using only one query, this time your dataframe
should not include the uid column, and the dataframe name is df_avg
- we want to test the hypothesis that the users who visited the newsfeed just a few
times have the lower delta between the first commit and the deadline. To do this,
you need to calculate the correlation coefficient between the number of pageviews
and the difference

    - using only one query, create a table with the columns: uid, avg_diff, pageviews
    - uid is the uids that exist in the test
    - avg_diff is the average delta between the first commit and the lab deadline per user
    - pageviews is the number of Newsfeed visits per user
    - do not take the lab ’project1’ into account
    - store it to the dataframe views_diff
    - use the Pandas method corr() to calculate the correlation coefficient between
    the number of pageviews and the difference


- close the connection

In [56]:
conn = sqlite3.connect('../data/checking-logs.sqlite')

In [3]:
schema_query = "PRAGMA table_info(test);"
schema = pd.io.sql.read_sql(schema_query, conn)
print(schema)

   cid             name       type  notnull dflt_value  pk
0    0              uid       TEXT        0       None   0
1    1          labname       TEXT        0       None   0
2    2  first_commit_ts       TEXT        0       None   0
3    3    first_view_ts  TIMESTAMP        0       None   0


In [17]:
query = "SELECT * FROM test LIMIT 10;"
pd.io.sql.read_sql(query, conn)

,uid,labname,first_commit_ts,first_view_ts
0,user_1,laba04,2020-04-26 17:06:18.462708,2020-04-26 21:53:59.624136
1,user_1,laba04s,2020-04-26 17:12:11.843671,2020-04-26 21:53:59.624136
2,user_1,laba05,2020-05-02 19:15:18.540185,2020-04-26 21:53:59.624136
3,user_1,laba06,2020-05-17 16:26:35.268534,2020-04-26 21:53:59.624136
4,user_1,laba06s,2020-05-20 12:23:37.289724,2020-04-26 21:53:59.624136
5,user_1,project1,2020-05-14 20:56:08.898880,2020-04-26 21:53:59.624136
6,user_10,laba04,2020-04-25 08:24:52.696624,2020-04-18 12:19:50.182714
7,user_10,laba04s,2020-04-25 08:37:54.604222,2020-04-18 12:19:50.182714
8,user_10,laba05,2020-05-01 19:27:26.063245,2020-04-18 12:19:50.182714
9,user_10,laba06,2020-05-19 11:39:28.885637,2020-04-18 12:19:50.182714


In [32]:
min_query = """
SELECT uid, min(diff)
FROM (SELECT test.uid, 
        CAST((julianday(test.first_commit_ts)- julianday(datetime(deadlines.deadlines, 'unixepoch'))) * 24 AS INTEGER) AS diff
    FROM test
        JOIN deadlines ON test.labname = deadlines.labs
    WHERE test.labname != 'project1');
"""
df_min = pd.io.sql.read_sql(min_query, conn)
df_min

,uid,min(diff)
0,user_30,-202


In [33]:
max_query = """
SELECT uid, max(diff)
FROM (SELECT test.uid, 
        CAST((julianday(test.first_commit_ts)- julianday(datetime(deadlines.deadlines, 'unixepoch'))) * 24 AS INTEGER) AS diff
    FROM test
        JOIN deadlines ON test.labname = deadlines.labs
    WHERE test.labname != 'project1') t;
"""
df_max = pd.io.sql.read_sql(max_query, conn)
df_max

,uid,max(diff)
0,user_25,-2


In [34]:
avg_query = """
SELECT uid, avg(diff)
FROM (SELECT test.uid, 
        CAST((julianday(test.first_commit_ts)- julianday(datetime(deadlines.deadlines, 'unixepoch'))) * 24 AS INTEGER) AS diff
    FROM test
        JOIN deadlines ON test.labname = deadlines.labs
    WHERE test.labname != 'project1') t;
"""
df_avg = pd.io.sql.read_sql(avg_query, conn)
df_avg

,uid,avg(diff)
0,user_1,-89.125


In [61]:
corr_query = """
SELECT uid, 
    AVG(diff) AS avg_diff,
    COUNT(uid) as pageviews 
FROM (SELECT test.uid, 
        CAST((julianday(test.first_commit_ts)- julianday(datetime(deadlines.deadlines, 'unixepoch'))) * 24 AS INTEGER) AS diff
    FROM test
        LEFT JOIN deadlines ON test.labname = deadlines.labs
    WHERE test.labname != 'project1') t
    JOIN pageviews USING(uid)
GROUP BY uid;
"""
views_diff = pd.io.sql.read_sql(corr_query, conn)
views_diff.corr(numeric_only=True)

,avg_diff,pageviews
avg_diff,1.000000,-0.185834
pageviews,-0.185834,1.000000


In [55]:
conn.close()